### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#create summary data frame with columns headers
district_summary_df = pd.DataFrame(columns =['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', \
'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing'])

In [3]:
#count of unique schools
district_summary_df['Total Schools'] = [school_data_complete['school_name'].nunique()]

#count of unique students counted and formated
district_summary_df['Total Students'] = [school_data_complete['Student ID'].nunique()]
district_summary_df['Total Students'] = district_summary_df['Total Students'].astype(float).map("{:,.0f}".format)

#total budget calculated formatted
district_summary_df['Total Budget'] = school_data['budget'].sum()
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].astype(float).map("${:,.2f}".format)

#average math score calculated
district_summary_df['Average Math Score'] = [school_data_complete['math_score'].mean()]

#average reading score calculated
district_summary_df['Average Reading Score'] = [school_data_complete['reading_score'].mean()]

#passing scores calculated
district_summary_df['% Passing Math'] = (school_data_complete.loc[school_data_complete['math_score'] >= 70]['Student ID'].nunique()) / school_data_complete['Student ID'].nunique() * 100
district_summary_df['% Passing Reading'] = (school_data_complete.loc[school_data_complete['reading_score'] >= 70]['Student ID'].nunique()) / school_data_complete['Student ID'].nunique() * 100
district_summary_df['% Overall Passing'] =  (school_data_complete.loc[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)]['Student ID'].nunique()) \
    / school_data_complete['Student ID'].nunique() * 100

#summary Data frame    
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
#create summary data frame with columns headers
school_summary_df = pd.DataFrame(columns =['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score', \
    'Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing'])


In [5]:
#Group new df by school_name
schools = school_data_complete.set_index('school_name').groupby(['school_name'])

#summary df with school names as index and pouplate type column
school_summary_df['School Type'] = school_data.set_index('school_name')['type']
school_summary_df = school_summary_df.sort_index(ascending=True)

#Total Students
school_summary_df['Total Students'] = schools['Student ID'].count()

#Total Budget calc
school_summary_df['Total School Budget'] = school_data.set_index('school_name')['budget']

#Total Budget per student calc
school_summary_df['Per Student Budget'] = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

#Average Math Score Calc
school_summary_df['Average Math Score'] = schools['math_score'].mean()

#Average Reading Score Calc
school_summary_df['Average Reading Score'] = schools['reading_score'].mean()

#Average % passing math Calc
school_summary_df['% Passing Math'] = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/schools['Student ID'].count() * 100

#Average % passing reading Calc
school_summary_df['% Passing Reading'] = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/schools['Student ID'].count() * 100

#Average % overall passing Calc
school_summary_df['% Overall Passing'] = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)] \
    .groupby('school_name')['Student ID'].count()/schools['Student ID'].count() * 100


#format school summary dataframe
school_summary_df.style.format({"Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:,.2f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.8}", 
                          "% Passing Reading": "{:.8}", 
                          "% Overall Passing": "{:.8}"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.93328,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
#dataframe for top schools
top_schools = school_summary_df.sort_values("% Overall Passing", ascending=False)

#format school summary dataframe
top_schools.head().style.format({"Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:,.2f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.8}", 
                          "% Passing Reading": "{:.8}", 
                          "% Overall Passing": "{:.8}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
#dataframe for top schools
bottom_schools = school_summary_df.sort_values("% Overall Passing", ascending=True)

#format school summary dataframe
bottom_schools.index.name=None
bottom_schools.head().style.format({"Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:,.2f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.8}", 
                          "% Passing Reading": "{:.8}", 
                          "% Overall Passing Rate": "{:.8}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
#Create pandas series for each grade.  Loc instead of conditional statement? 
ninth_grade = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_grade = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_grade = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_grade = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

#comebine series into a dataframe
mathscores_grade = pd.DataFrame({
        "9th": ninth_grade,
        "10th": tenth_grade,
        "11th": eleventh_grade,
        "12th": twelfth_grade
})

#Optional: give the displayed data cleaner format
mathscores_grade.index.name=None
mathscores_grade.style.format({'9th': '{:.2f}', 
                          "10th": '{:.2f}', 
                          "11th": "{:.2f}", 
                          "12th": "{:.2f}"})




,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
#Create pandas series for each grade.  Loc instead of conditional statement? 
ninth_grade = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_grade = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_grade = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_grade = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

#comebine series into a dataframe
readscores_grade = pd.DataFrame({
        "9th": ninth_grade,
        "10th": tenth_grade,
        "11th": eleventh_grade,
        "12th": twelfth_grade
})

#Optional: give the displayed data cleaner format
readscores_grade.index.name=None
readscores_grade.style.format({'9th': '{:.2f}', 
                          "10th": '{:.2f}', 
                          "11th": "{:.2f}", 
                          "12th": "{:.2f}"})

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# Create bins in which to place values based upon IMDB vote count
bins = [0, 585, 630, 645, 680]

# Create labels for these bins
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#add % passing rate and bins to dataframe
school_data_complete['Spending Ranges (Per Student)'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], bins, labels = labels)

#group data to bins
grouped_data_spend = school_data_complete.groupby('Spending Ranges (Per Student)')

#create summary data frame with columns headers
scores_by_schoolspend = pd.DataFrame(columns =["Average Math Score","Average Reading Score",'% Passing Math','% Passing Reading',"% Overall Passing Rate"])

scores_by_schoolspend['Average Math Score'] = grouped_data_spend['math_score'].mean()
scores_by_schoolspend['Average Reading Score'] = grouped_data_spend['reading_score'].mean()
scores_by_schoolspend['% Passing Math'] = school_data_complete[school_data_complete['math_score'] >= 70].groupby('Spending Ranges (Per Student)')['Student ID'].count()/grouped_data_spend['Student ID'].count() * 100
scores_by_schoolspend['% Passing Reading'] = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('Spending Ranges (Per Student)')['Student ID'].count()/grouped_data_spend['Student ID'].count() * 100
scores_by_schoolspend['% Overall Passing Rate'] = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('Spending Ranges (Per Student)')['Student ID'].count()/grouped_data_spend['Student ID'].count() *100

#give the displayed data cleaner format
scores_by_schoolspend.style.format({'Average Math Score': '{:.2f}', 
                          "Average Reading Score": '{:.2f}', 
                          "% Passing Math": "{:.2f}", 
                          "% Passing Reading": "{:.2f}",
                          "% Overall Passing Rate": "{:.2f}",
                          })


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.36,83.96,93.70,96.69,90.64
$585-630,79.98,82.31,79.11,88.51,70.94
$630-645,77.82,81.30,70.62,82.60,58.84
$645-680,77.05,81.01,66.23,81.11,53.53


* Perform the same operations as above, based on school size.

In [11]:
# Create bins in which to place values based upon IMDB vote count
bins = [0, 1000, 2000, 5000]

# Create labels for these bins
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
school_summary_df["School Size"] = pd.cut(school_summary_df['Total Students'], bins, labels = labels)

# Calculate averages for the desired columns. 
size_math_scores = school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

#create summary data frame with columns headers
size_summary = pd.DataFrame(columns =["Average Math Score","Average Reading Score",'% Passing Math','% Passing Reading',"% Overall Passing Rate"])

#assign series to dataframe 
size_summary['Average Math Score'] = size_math_scores
size_summary['Average Reading Score'] = size_reading_scores
size_summary['% Passing Math'] = size_passing_math
size_summary['% Passing Reading'] = size_passing_reading
size_summary['% Overall Passing Rate'] = size_overall_passing

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [13]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
by_type = school_summary_df.groupby('School Type').mean()

# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame(columns =["Average Math Score","Average Reading Score",'% Passing Math','% Passing Reading',"% Overall Passing Rate"])

#assign series to dataframe 
type_summary['Average Math Score'] = by_type["Average Math Score"]
type_summary['Average Reading Score'] = by_type['Average Reading Score']
type_summary['% Passing Math'] = by_type['% Passing Math']
type_summary['% Passing Reading'] = by_type['% Passing Reading']
type_summary['% Overall Passing Rate'] = by_type['% Overall Passing']

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
